In [1]:
import re
import pandas as pd 
import numpy as np
import nltk
import warnings
import pickle
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import data_cleaner

In [3]:
df_train = pd.read_csv('data/train.csv')
df_t = df_train.copy()

In [4]:
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
df_train['clean_tweet'] = np.vectorize(data_cleaner.clean_mentions)(df_train['tweet'])
df_train['clean_tweet'] = df_train['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
df_train['clean_tweet'] = df_train['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))


df_train.head()

C:\Users\Tomiu\AppData\Local\Temp\ipykernel_16736\3053594222.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train['clean_tweet'] = df_train['clean_tweet'].str.replace("[^a-zA-Z#]", " ")


,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when father dysfunctional and selfish drags hi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit can use cause they don...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model love take with all the time
4,5,0,factsguide: society now #motivation,factsguide society now #motivation


In [11]:
from data_transformation import TfidfDataTransformer
from sklearn.model_selection import train_test_split

tr= TfidfDataTransformer()

df_train['clean_tweet'] = tr.stemming(df_train['clean_tweet'])
display(df_train.head())
df_train['clean_tweet'].to_csv('../models/model_data.csv', sep=',')
df_tfidf = tr.fit_transform(df_train['clean_tweet'])
pickle.dump(tr, open('../models/vectorizer.sav', 'wb'))
x_train, x_valid, y_train, y_valid = train_test_split(df_tfidf, df_train['label'],test_size=0.3,random_state=42)

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when father dysfunct and selfish drag hi kid i...
1,2,0,@user @user thanks for #lyft credit i can't us...,thank for #lyft credit can use cau they don of...
2,3,0,bihday your majesty,bihday your majesti
3,4,0,#model i love u take with u all the time in ...,#model love take with all the time
4,5,0,factsguide: society now #motivation,factsguid societi now #motiv


# Models

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
model = LogisticRegression(random_state=0,solver='lbfgs')
model.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [11]:
predictions = model.predict_proba(x_valid)
predictions

array([[0.99002625, 0.00997375],
       [0.98121538, 0.01878462],
       [0.91198796, 0.08801204],
       ...,
       [0.94022065, 0.05977935],
       [0.50609832, 0.49390168],
       [0.97454842, 0.02545158]])

In [12]:
prediction_int = predictions[:,1]>=0.3
prediction_int = prediction_int.astype(np.int)
log_tfidf = f1_score(y_valid, prediction_int)

log_tfidf

0.5426356589147286

In [14]:
pickle.dump(model, open('../models/model.sav', 'wb'))
 

In [55]:
model.predict_proba(['This thing is pretty shit'])


ValueError: Expected 2D array, got 1D array instead:
array=['This thing is pretty shit'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.